# Elastic, Viscoplastic model with nonlinear hardening

Purpose - demonstrate the limits of non-damage model for hysteretic fatigue modeling 

By introducing a nonlinear isotropic model, it is possible to model the monotonic response of a quasi-brittle material with a softening response after achieving the stress limit. 

Show the role of the individual dissipative mechanisms, including viscoelastic damage, isotropic and kinematic hardening, and damage. 

Plan

 - reproduce the monotonic response using e.g. an exponential function analogical to Weibull probability density function.
 - define a cyclic loading which touches the elastic limits both at the upper and lower load levels
 - show that the strain does not increase 


In [ ]:
%matplotlib widget
from bmcs_matmod import GSM
import matplotlib.pylab as plt 
import sympy as sp
import numpy as np
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_d_likh import Potential1D_T_E_VP_D_LIKH_SymbExpr
sp.init_printing()

In [ ]:
p1d = Potential1D_T_E_VP_D_LIKH_SymbExpr()
p1d.F_

In [ ]:
eps_a = p1d.eps_a
sig_a = p1d.sig_a
dF_du = p1d.F_.diff(eps_a)
# dF_du = dF_du.xreplace({h: 0 for h in dF_du.atoms(sp.DiracDelta)})
# dF_du = dF_du.xreplace({h: 1 for h in dF_du.atoms(sp.Heaviside)})
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, eps_a, dF_du)])
subs_u_sig_ = dict(zip(eps_a, u_sig_))
subs_u_sig_

In [ ]:
sig_x_u_ = (sig_a.T * eps_a)[0]
G_expr = sig_x_u_ - p1d.F_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

In [ ]:
gsm_F = GSM(
    name = 'gsm_F_1d_t_e_vp_d_likh',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

gsm_G = GSM(
    name = 'gsm_G_1d_t_e_vp_d_likh',
    u_vars = p1d.sig_a,
    sig_vars = p1d.eps_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = G_,
    dF_sign = -1,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

In [ ]:
gsm_F.save_to_disk()
gsm_G.save_to_disk()

In [ ]:
gsm_F.Eps_vars

In [ ]:
gsm_F.F_expr

In [ ]:
gsm_F.phi_

In [ ]:
gsm_F.dF_dEps_

In [ ]:
gsm_F.Sig_vars, gsm_F.Sig_

In [ ]:
gsm_F.dSig_dEps_

In [ ]:
p1d.f_

In [ ]:
gsm_F.df_dSig_

In [ ]:
gsm_F.df_dEps_

In [ ]:
gsm_F.Phi_

In [ ]:
gsm_F.df_dlambda_

In [ ]:
gsm_F.m_params